<a href="https://colab.research.google.com/github/Dulavinya/Tea_defect_detection/blob/main/Yield_quality_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# 3. Install Ultralytics ###
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.1 MB/s eta 0:00:00


In [2]:


!pip install pyyaml

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
zip_path = '/content/drive/MyDrive/Tea growth stage.v3i.yolov8.zip'

!unzip -q "{zip_path}" -d ./FER

In [6]:
import yaml



file_path = './FER/data.yaml'
with open(file_path, 'r') as file:
    data = yaml.safe_load(file)

print(data)

{'train': '../train/images', 'val': '../valid/images', 'test': '../test/images', 'nc': 3, 'names': ['high', 'law', 'medium'], 'roboflow': {'workspace': 'children-and-adults-detection', 'project': 'tea-growth-stage', 'version': 3, 'license': 'CC BY 4.0', 'url': 'https://universe.roboflow.com/children-and-adults-detection/tea-growth-stage/dataset/3'}}


In [7]:
classes=data["names"]
print(classes)

['high', 'law', 'medium']


In [8]:


dict_classes={}
for i, j in enumerate(classes):
  dict_classes[i]=j
print(dict_classes)


updated_dict = {key: int(value) if value.isdigit() else value for key, value in dict_classes.items()}

print(updated_dict)

{0: 'high', 1: 'law', 2: 'medium'}
{0: 'high', 1: 'law', 2: 'medium'}


In [9]:
%%writefile './FER/growth_stage_detection_config.yaml'

path: './FER' # dataset root dir
train: train # train images (relative to 'path')
val: valid  # val images (relative to 'path')

names:
  0: 'high'
  1: 'law'
  2: 'medium'




Writing ./FER/growth_stage_detection_config.yaml


In [11]:
### 2. Define root directory ###
root_dir= './FER'

In [14]:
import torch
torch.cuda.is_available()  # should now return True
torch.cuda.device_count()  # should return 1

0

In [13]:
### 4. Train model ###

import os

from ultralytics import YOLO


# Load a model
model = YOLO("yolov8s.pt")  # load pre trained model

# Use the model
results = model.train(data=os.path.join(root_dir, "growth_stage_detection_config.yaml"), epochs=100,imgsz=640,patience=50,batch=0.70,device=0,
                      degrees=90)  # train the model

Ultralytics 8.3.186 🚀 Python-3.12.11 torch-2.8.0+cu126 


ValueError: Invalid CUDA 'device=0' requested. Use 'device=cpu' or pass valid CUDA device(s) if available, i.e. 'device=0' or 'device=0,1,2,3' for Multi-GPU.

torch.cuda.is_available(): False
torch.cuda.device_count(): 0
os.environ['CUDA_VISIBLE_DEVICES']: 0
See https://pytorch.org/get-started/locally/ for up-to-date torch install instructions if no CUDA devices are seen by torch.
